<a href="https://colab.research.google.com/github/JstnClmnt/NLP-Text-Summarization/blob/master/NLP_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import re
np.random.seed(1)

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os
 
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = dirName+"/"+entry
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles        

In [6]:
!unzip "/content/drive/Team Drives/AI Lords/NLP-Text-Summarization/dataset.zip"

Archive:  /content/drive/Team Drives/AI Lords/NLP-Text-Summarization/dataset.zip
   creating: data/
   creating: data/business/
  inflating: data/business/001.txt   
  inflating: data/business/002.txt   
  inflating: data/business/003.txt   
  inflating: data/business/004.txt   
  inflating: data/business/005.txt   
  inflating: data/business/006.txt   
  inflating: data/business/007.txt   
  inflating: data/business/008.txt   
  inflating: data/business/009.txt   
  inflating: data/business/010.txt   
  inflating: data/business/011.txt   
  inflating: data/business/012.txt   
  inflating: data/business/013.txt   
  inflating: data/business/014.txt   
  inflating: data/business/015.txt   
  inflating: data/business/016.txt   
  inflating: data/business/017.txt   
  inflating: data/business/018.txt   
  inflating: data/business/019.txt   
  inflating: data/business/020.txt   
  inflating: data/business/021.txt   
  inflating: data/business/022.txt   
  inflating: data/business/023.txt  

In [7]:
import pandas as pd


dirName = 'data';
    
# Get the list of all files in directory tree at given path
listOfFiles = getListOfFiles(dirName)
    

df=pd.DataFrame(columns=["Title","Description","Category"])
# Print the files    
for elem in listOfFiles:
    print("Reading: "+elem)
    file1 = open(elem,"r",encoding="latin-1") 
    sampleNews=file1.read().split("\n")
    newsDesc=""
    for strline in sampleNews[1:len(sampleNews)]:
        newsDesc+=strline
    """
    print("Category: "+elem.split("/")[1])
    print("Title:"+sampleNews[0])
    print("Description:"+newsDesc[0:50])
    """
    dfsample=pd.DataFrame(columns=["Title","Description","Category"],data=[[sampleNews[0],newsDesc,elem.split("/")[1]]])
    df=df.append(dfsample)

Reading: data/entertainment/303.txt
Reading: data/entertainment/235.txt
Reading: data/entertainment/162.txt
Reading: data/entertainment/200.txt
Reading: data/entertainment/244.txt
Reading: data/entertainment/109.txt
Reading: data/entertainment/029.txt
Reading: data/entertainment/227.txt
Reading: data/entertainment/139.txt
Reading: data/entertainment/166.txt
Reading: data/entertainment/320.txt
Reading: data/entertainment/191.txt
Reading: data/entertainment/032.txt
Reading: data/entertainment/170.txt
Reading: data/entertainment/280.txt
Reading: data/entertainment/301.txt
Reading: data/entertainment/263.txt
Reading: data/entertainment/045.txt
Reading: data/entertainment/276.txt
Reading: data/entertainment/176.txt
Reading: data/entertainment/206.txt
Reading: data/entertainment/104.txt
Reading: data/entertainment/100.txt
Reading: data/entertainment/077.txt
Reading: data/entertainment/051.txt
Reading: data/entertainment/186.txt
Reading: data/entertainment/215.txt
Reading: data/entertainment/

In [8]:
df=df.reset_index()
df=df.drop(list(df)[0], axis=1)
df["Description"].head()[0]

'The number of British films produced in the UK fell 40% in 2004, with movies starring Johnny Depp and Jude Law among those affected, Screen Daily has said.Twenty-seven British films were made in the UK last year following a drop in funding, compared with 45 in 2003, according to the trade weekly. It attributed the drop to tighter tax laws and reduced funding from sources such as the National Lottery. UK and US co-productions in Britain fell from 102 in 2003 to 81 last year.Last year the government closed tax loopholes that many film investors had taken advantage of to fund films in the UK. Several projects were shelved as they faced the implications of this clampdown, coupled with the reduction in movie funding from traditional sources including the lottery and Miramax Films. Shooting on period drama Tulip Fever, which was to star Jude Law and Keira Knightley, was postponed indefinitely while shooting of The Libertine starring Johnny Depp was moved to the Isle of Man."There isn\'t any

In [9]:
df=df[["Title","Description"]]
df.head()

,Title,Description
0,Film production 'falls' 40% in UK,The number of British films produced in the UK...
1,Pop band Busted to 'take a break',Chart-topping pop band Busted have confirmed t...
2,Prince crowned 'top music earner',Prince earned more than any other pop star in ...
3,Celebrities get to stay in jungle,All four contestants still remain in I'm A Cel...
4,Ten-year tragedy of missing Manic,"Richey Edwards, guitarist and lyricist for The..."


In [0]:
df['Title'] = df['Title'].str.replace("[^a-zA-Z#]"," ")
df['Description'] = df['Description'].str.replace("[^a-zA-Z#]"," ")
df=df.dropna()

In [11]:
from keras.preprocessing.text import Tokenizer
VOCAB_SIZE = 20000
tokenizer = Tokenizer(oov_token=1,num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(df["Description"])
article_sequences = tokenizer.texts_to_sequences(df["Description"])
art_word_index = tokenizer.word_index
len(art_word_index)


Using TensorFlow backend.


27918

In [12]:
art_word_index_1500 = {}
counter = 0
for word in art_word_index.keys():
    if art_word_index[word] == 0:
        print("found 0!")
        break
    if art_word_index[word] > VOCAB_SIZE:
        continue
    else:
        art_word_index_1500[word] = art_word_index[word]
        counter += 1
print(counter)


20000


In [13]:
tokenizer.fit_on_texts(df["Title"])
summary_sequences = tokenizer.texts_to_sequences(df["Title"])
sum_word_index = tokenizer.word_index
len(sum_word_index)

28062

In [14]:
sum_word_index_1500 = {}
counter = 0
for word in sum_word_index.keys():
    if sum_word_index[word] == 0:
        print("found 0!")
        break
    if sum_word_index[word] > VOCAB_SIZE:
        continue
    else:
        sum_word_index_1500[word] = sum_word_index[word]
        counter += 1
print(counter)


20000


In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(article_sequences, summary_sequences, test_size=0.20, random_state=42)

print(X_train[0])

[3073, 4796, 19, 14, 31, 6276, 4, 6, 2213, 7, 2, 370, 4, 956, 24, 528, 128, 3, 223, 2476, 383, 1330, 18, 2927, 3624, 1407, 13, 2, 112, 8, 3619, 18, 2906, 12, 15, 190, 3, 33, 4796, 11, 16, 37, 501, 5429, 3, 956, 530, 20, 1, 1, 1, 5, 12534, 25, 2, 8049, 173, 1873, 14, 31, 861, 97, 76, 81, 448, 499, 3, 17, 190, 7, 4161, 16, 14, 4770, 9, 1407, 499, 3, 17, 1501, 2, 956, 3, 563, 9, 12, 12, 15, 2117, 56, 3, 194, 11, 61, 2, 2558, 3624, 1407, 10, 131, 3, 117, 231, 12, 10, 131, 3, 2850, 4770, 7, 2, 956, 5, 48, 52, 2113, 3, 563, 9, 12, 33, 4796, 118, 11, 2, 141, 1985, 457, 31, 6276, 4, 2, 17238, 15, 31, 3365, 20, 467, 4, 2, 166, 526, 855, 6050, 11, 12, 8, 8765, 1928, 9, 2, 166, 526, 3, 21, 43, 467, 7, 2, 370, 4, 956, 5, 11, 15, 6, 13822, 1985, 33, 4796, 14, 16, 394, 54, 2, 275, 9, 744, 6, 207, 69, 104, 16, 37, 207, 37, 5943, 54, 2, 956, 25, 16, 118, 2, 840, 4, 1407, 19, 1288, 4713, 25, 29, 8886, 5, 16148, 27, 167, 117, 60, 177, 3127, 78, 4976, 104, 12, 8, 29, 6, 275, 3, 17, 97, 1, 2, 205, 110, 34

In [16]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2019-05-21 14:16:58--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-05-21 14:16:59--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  8.88MB/s    in 74s     

2019-05-21 14:18:13 (11.1 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [17]:
!unzip "glove.6B.zip"

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
import numpy as np

embeddings_index = dict()
f = open('glove.6B.300d.txt',encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [0]:
EMB_DIM=300
num_words=VOCAB_SIZE+1
print("Number of Words:"+str(num_words))

#MAX_LENGTH = len(max(X_train, key=len))
#print("Max Length: "+str(MAX_LENGTH))  # 271

MAX_LENGTH = 500
print("Max Length for Sentences: "+str(MAX_LENGTH))
MAX_LENGTH_SUM = len(max(y_train, key=len))
print("Max Length for Summaries: "+str(MAX_LENGTH_SUM))  # 271

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
train_sentences_X = pad_sequences(X_train, maxlen=MAX_LENGTH, padding='post',truncating="post")
test_sentences_X = pad_sequences(X_test, maxlen=MAX_LENGTH, padding='post',truncating="post")
train_sentences_Y= pad_sequences(y_train, maxlen=MAX_LENGTH_SUM, padding='post',truncating="post")
test_sentences_Y= pad_sequences(y_test, maxlen=MAX_LENGTH_SUM, padding='post',truncating="post")
#y_train=to_categorical(y_train)
#y_test=to_categorical(y_test)
print(train_sentences_X[0])
print(train_sentences_Y[0])

Number of Words:20001
Max Length for Sentences: 500
Max Length for Summaries: 9
[ 6643  2428    51     4     2    71     8   724  2428  1468    63   375
    44     3   508    64     2   109   168    81    43   201   302    19
    14     2  3056   218   123    10   353     3  1895    43    89  3274
     4    53   149 17503   109   221   281    12    51     4     2   724
   562   238     4    43  1237   175     1  6643    19    92   834     2
  2402   123    24    21     3  5404  1085     4   508     2  1043  1473
  9717   167    29   151   178     2   495  1902    38   701    33  6643
    92    53   834    11   501     2  3274     4   228  2428   184    15
  2144     2   123    38  1787     3  1424    43  3358    23   162     5
  6840    34    63   165    43  3358 13921    28     3   866    23    32
    26  4887     7 17271    74  2032    24   144   135    33  6643    92
     2   684     7  2938  6643  2428    15  3469    65    42    67    33
  6643     8 13878  4597  5168    18  2086  

In [24]:
EMB_DIM=300
num_words=VOCAB_SIZE+1
embedding_matrix_art=np.zeros((num_words,EMB_DIM))
numNoEmb=0
#print(word2index)
for word,i in art_word_index_1500.items():
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_art[i]=embedding_vector
    else:
      numNoEmb+=1
print(numNoEmb)

424


In [25]:
embedding_matrix_sum=np.zeros((num_words,EMB_DIM))
#print(word2index)
numNoEmb=0
for word,i in sum_word_index_1500.items():
    if i>num_words:
        continue
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_sum[i]=embedding_vector
    else:
      numNoEmb+=1
print(numNoEmb)

413


In [28]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import concatenate,Input,Dense, CuDNNLSTM,LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation,Dropout,Add
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras import regularizers
from tensorflow.keras.initializers import Constant
import numpy as np
from tensorflow.keras.utils import plot_model

HIDDEN_UNITS=512
MAX_LENGTH = 500
MAX_LENGTH_SUM = len(max(y_train, key=len))

"""
Chatbot Inspired Encoder-Decoder-seq2seq
"""
'''
encoder_inputs = Input(shape=(MAX_LENGTH, ))
encoder_embedding = Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix_art),input_length=MAX_LENGTH,trainable=False,mask_zero=True)(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

decoder_inputs = Input(shape=(MAX_LENGTH, ))
decoder_embedding = Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix_sum),input_length=MAX_LENGTH,trainable=False,mask_zero=True)(decoder_inputs)
decoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])

# dense_layer = Dense(VOCAB_SIZE, activation='softmax')
outputs = TimeDistributed(Dense(num_words, activation='softmax'))(decoder_outputs)
model = tf.keras.Model([encoder_inputs, decoder_inputs], outputs)
'''


"""
Bidirectional LSTM: Others Inspired Encoder-Decoder-seq2seq
"""
encoder_inputs = Input(shape=(MAX_LENGTH,))
encoder_embedding = Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix_art),input_length=MAX_LENGTH,trainable=False,mask_zero=True)(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True)
encoder_LSTM_R = LSTM(HIDDEN_UNITS, return_state=True, go_backwards=True)
encoder_outputs_R, state_h_R, state_c_R = encoder_LSTM_R(encoder_embedding)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

final_h = Add()([state_h, state_h_R])
final_c = Add()([state_c, state_c_R])
encoder_states = [final_h, final_c]

"""
decoder
"""
decoder_inputs = Input(shape=(MAX_LENGTH_SUM,))
decoder_embedding = decoder_embedding = Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix_sum),input_length=MAX_LENGTH_SUM,trainable=False,mask_zero=True)(decoder_inputs)
decoder_LSTM = LSTM(HIDDEN_UNITS, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=encoder_states) 
decoder_dense = TimeDistributed(Dense(num_words, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

model= tf.keras.Model(inputs=[encoder_inputs,decoder_inputs], outputs=decoder_outputs)


model.compile(optimizer=Adam(0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

print(model.summary())
plot_model(model, to_file='model_seq2seq.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 500, 300)     6000300     input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 9)            0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 512), (None, 1665024     embedding_1[0][0]                
__________________________________________________________________________________________________
lstm_3 (LS

In [31]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_sentences_Y= pad_sequences(y_train, maxlen=MAX_LENGTH_SUM, padding='post',truncating="post")

num_samples_train = len(train_sentences_Y)
print(num_samples_train)
decoder_output_data_train = np.zeros(shape=(num_samples_train, MAX_LENGTH_SUM, num_words))

1780


In [33]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

test_sentences_Y= pad_sequences(y_test, maxlen=MAX_LENGTH_SUM, padding='post',truncating="post")

num_samples_test = len(test_sentences_Y)
print(num_samples_test)
decoder_output_data_test = np.zeros(shape=(num_samples_test, MAX_LENGTH_SUM, num_words))

445


In [0]:
for i, seqs in enumerate(train_sentences_Y):
    for j, seq in enumerate(seqs):
        if j > 0:
            decoder_output_data_train[i][j][seq] = 1
            
for i, seqs in enumerate(test_sentences_Y):
    for j, seq in enumerate(seqs):
        if j > 0:
            decoder_output_data_test[i][j][seq] = 1

In [38]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences

test_sentences_X = pad_sequences(X_test, maxlen=MAX_LENGTH, padding='post',truncating="post")
train_sentences_X = pad_sequences(X_train, maxlen=MAX_LENGTH, padding='post',truncating="post")

save = ModelCheckpoint('best_model.hdf5', save_best_only=True, monitor='acc', mode='max')
es_acc = EarlyStopping(monitor='acc', mode='max',verbose=1,patience=10,min_delta=0.003)
es_loss = EarlyStopping(monitor='loss', mode='min',patience=10,verbose=1)
history = model.fit([train_sentences_X, train_sentences_Y], 
                     decoder_output_data_train, 
                     epochs=100, 
                     batch_size=32,
                     validation_data=([test_sentences_X, test_sentences_Y], decoder_output_data_test),
                     callbacks=[save,es_acc,es_loss])

Train on 1780 samples, validate on 445 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
1780/1780 [==============================] - 216s 121ms/sample - loss: 7.6447 - acc: 0.0259 - val_loss: 6.7091 - val_acc: 0.0276
Epoch 2/100
1780/1780 [==============================] - 210s 118ms/sample - loss: 5.9910 - acc: 0.0299 - val_loss: 6.5916 - val_acc: 0.0276
Epoch 3/100
1780/1780 [==============================] - 195s 110ms/sample - loss: 5.8111 - acc: 0.0299 - val_loss: 6.6707 - val_acc: 0.0276
Epoch 4/100
1780/1780 [==============================] - 195s 110ms/sample - loss: 5.7701 - acc: 0.0299 - val_loss: 6.7179 - val_acc: 0.0276
Epoch 5/100
1780/1780 [==============================] - 194s 109ms/sample - loss: 5.7464 - acc: 0.0299 - val_loss: 6.7672 - val_acc: 0.0276
Epoch 6/100
1780/1780 [==============================] - 196s 110ms/sample - loss: 5.7272 - acc: 0.0299 - val_loss: 6.7951 - val_acc: 0.0276
Epoch 7/100
1780/1780 [==============================] - 19

In [1]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure()
plt.plot(history_lstm.history['acc'])
plt.plot(history_lstm.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.yticks(np.arange(0,1,step=0.1))
plt.savefig("acc_lstm.png")

plt.figure( )
# Plot training & validation loss values
plt.plot(history_lstm.history['loss'])
plt.plot(history_lstm.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig("loss_lstm.png")

NameError: ignored

<Figure size 432x288 with 0 Axes>